# Item-Based Collaborative Filtering with KNN
Bu notebook MovieLens veri seti üzerinde item-based collaborative filtering uygulamaktadır.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Veri dosyalarını yükle
ratings_path = "/kaggle/input/movielens-1m-dataset/ratings.dat"
movies_path = "/kaggle/input/movielens-1m-dataset/movies.dat"
users_path = "/kaggle/input/movielens-1m-dataset/users.dat"


# Ratings dataseti
ratings = pd.read_csv(ratings_path, sep='::', header=None, 
                       names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
                       engine='python', encoding='latin-1')

# Movies dataseti
movies = pd.read_csv(movies_path, sep='::', header=None, 
                      names=['MovieID', 'Title', 'Genres'],
                      engine='python', encoding='latin-1')

# Users dataseti
users = pd.read_csv(users_path, sep='::', header=None,
                     names=['UserID', 'Gender', 'Age', 'Occupation', 'ZipCode'],
                     engine='python', encoding='latin-1')

print("Ratings Shape:", ratings.shape)
print("\nRatings ilk 5 satır:")
print(ratings.head())
print("\n" + "="*50)
print("Movies Shape:", movies.shape)
print("Movies ilk 5 satır:")
print(movies.head())

Ratings Shape: (1000209, 4)

Ratings ilk 5 satır:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Movies Shape: (3883, 3)
Movies ilk 5 satır:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [2]:
# Veri analizi
print("VERI ANALIZI")
print("="*50)
print(f"Toplam kullanıcı: {ratings['UserID'].nunique()}")
print(f"Toplam film: {ratings['MovieID'].nunique()}")
print(f"Toplam rating: {len(ratings)}")
print(f"\nRating istatistikleri:")
print(ratings['Rating'].describe())
print(f"\nRating dağılımı:")
print(ratings['Rating'].value_counts().sort_index())

VERI ANALIZI
Toplam kullanıcı: 6040
Toplam film: 3706
Toplam rating: 1000209

Rating istatistikleri:
count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

Rating dağılımı:
Rating
1     56174
2    107557
3    261197
4    348971
5    226310
Name: count, dtype: int64


In [3]:
# User-Item matrisini oluştur
user_item_matrix = ratings.pivot_table(index='UserID', 
                                        columns='MovieID', 
                                        values='Rating',
                                        fill_value=0)

print("User-Item Matrix Shape:", user_item_matrix.shape)
print("İlk 5x5 User-Item Matrix:")
print(user_item_matrix.iloc[:5, :5])
print(f"\nMatrix sparsity: {(user_item_matrix == 0).sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100:.2f}%")

User-Item Matrix Shape: (6040, 3706)
İlk 5x5 User-Item Matrix:
MovieID    1    2    3    4    5
UserID                          
1        5.0  0.0  0.0  0.0  0.0
2        0.0  0.0  0.0  0.0  0.0
3        0.0  0.0  0.0  0.0  0.0
4        0.0  0.0  0.0  0.0  0.0
5        0.0  0.0  0.0  0.0  0.0

Matrix sparsity: 95.53%


In [4]:
# Train-test split (80-20)
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

print(f"Eğitim seti boyutu: {len(train_ratings)}")
print(f"Test seti boyutu: {len(test_ratings)}")

# Eğitim seti için user-item matrix
train_user_item = train_ratings.pivot_table(index='UserID',
                                             columns='MovieID',
                                             values='Rating',
                                             fill_value=0)

print(f"\nTrain User-Item Matrix Shape: {train_user_item.shape}")

Eğitim seti boyutu: 800167
Test seti boyutu: 200042

Train User-Item Matrix Shape: (6040, 3683)


In [5]:
# Item-based KNN Collaborative Filtering
class ItemBasedKNN:
    def __init__(self, n_neighbors=5, metric='cosine'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.train_matrix = None
        self.knn_model = None
        self.item_similarity = None
        
    def fit(self, train_matrix):
        """
        KNN modelini eğit
        train_matrix: User-Item matrix (Users x Items)
        """
        self.train_matrix = train_matrix.copy()
        
        # Item bazlı komşuluğu bul (Items x Items)
        # Transpose et: Items x Users
        item_matrix = train_matrix.T
        
        # KNN modeli oluştur
        self.knn_model = NearestNeighbors(n_neighbors=self.n_neighbors + 1,  # +1 çünkü kendisini içerir
                                          metric=self.metric,
                                          n_jobs=-1)
        self.knn_model.fit(item_matrix)
        
        # Item similarity matrisini hesapla
        distances, indices = self.knn_model.kneighbors(item_matrix)
        
        # Similarity = 1 / (1 + distance)
        self.item_similarity = {}
        for i in range(len(item_matrix)):
            item_id = train_matrix.columns[i]
            similar_items = {}
            for dist, idx in zip(distances[i][1:], indices[i][1:]):  # [1:] kendisini hariç tut
                similar_item_id = train_matrix.columns[idx]
                similarity = 1 / (1 + dist)
                similar_items[similar_item_id] = similarity
            self.item_similarity[item_id] = similar_items
        
        return self
    
    def predict(self, user_id, item_id):
        """
        Bir kullanıcı ve film için rating tahmin et
        """
        if user_id not in self.train_matrix.index or item_id not in self.train_matrix.columns:
            return self.train_matrix.values.mean()  # Ortalama rating döndür
        
        # Benzer filmler al
        if item_id not in self.item_similarity:
            return self.train_matrix.values.mean()
        
        similar_items = self.item_similarity[item_id]
        
        if not similar_items:
            return self.train_matrix.values.mean()
        
        # Kullanıcının benzer filmlere verdiği ratingler
        user_ratings = self.train_matrix.loc[user_id]
        
        numerator = 0
        denominator = 0
        
        for sim_item_id, similarity in similar_items.items():
            user_rating = user_ratings[sim_item_id]
            if user_rating > 0:  # Kullanıcı bu filmi izlemişse
                numerator += similarity * user_rating
                denominator += similarity
        
        if denominator > 0:
            return numerator / denominator
        else:
            return self.train_matrix.values.mean()

print("ItemBasedKNN modeli tanımlandı")

ItemBasedKNN modeli tanımlandı


In [6]:
# KNN modelini eğit - PARAMETRE OPTİMİZASYONU (k=3,5,7,10,15,20,30 - FULL TEST SET)
print("KNN modeli eğitiliyor...")
print("="*80)

# Tüm test setini kullan
test_sample = test_ratings.copy()
print(f"Test seti: {len(test_sample)} örnek (tüm veri seti)\n")

# Spesifik k değerlerini test et
k_values = [3, 5, 7, 10, 15, 20, 30]
results = []

for k in k_values:
    print(f"{'='*60}")
    print(f"k = {k} ile eğitim başlıyor...")
    print(f"{'='*60}")
    
    knn_cf = ItemBasedKNN(n_neighbors=k, metric='cosine')
    knn_cf.fit(train_user_item)
    
    print(f"k = {k} tahminleri hesaplanıyor ({len(test_sample)} örnek)...")
    
    # Test setinde tahminler yap
    predictions_temp = []
    actuals_temp = []
    valid_count = 0
    processed = 0
    
    for idx, row in test_sample.iterrows():
        user_id = row['UserID']
        movie_id = row['MovieID']
        actual_rating = row['Rating']
        
        # Sadece train setinde bulunan user ve movie'leri tahmin et
        if user_id in knn_cf.train_matrix.index and movie_id in knn_cf.train_matrix.columns:
            predicted_rating = knn_cf.predict(user_id, movie_id)
            predicted_rating = np.clip(predicted_rating, 1, 5)
            
            predictions_temp.append(predicted_rating)
            actuals_temp.append(actual_rating)
            valid_count += 1
        
        processed += 1
        # Her 50000 tahminde ilerlemesi göster
        if processed % 50000 == 0:
            print(f"  {processed} örnek işlendi... ({valid_count} geçerli tahmin)")
    
    if valid_count > 0:
        predictions_temp = np.array(predictions_temp)
        actuals_temp = np.array(actuals_temp)
        
        # Metrikleri hesapla
        rmse_temp = np.sqrt(mean_squared_error(actuals_temp, predictions_temp))
        mae_temp = mean_absolute_error(actuals_temp, predictions_temp)
        r2_temp = 1 - (np.sum((actuals_temp - predictions_temp)**2) / np.sum((actuals_temp - actuals_temp.mean())**2))
        
        results.append({
            'k': k,
            'RMSE': rmse_temp,
            'MAE': mae_temp,
            'R2': r2_temp,
            'Valid_Count': valid_count
        })
        
        print(f"\n✓ k={k} Sonuçları:")
        print(f"   RMSE: {rmse_temp:.4f} | MAE: {mae_temp:.4f} | R²: {r2_temp:.4f}")
        print(f"   Geçerli tahmin: {valid_count}\n")
    else:
        print(f"  ✗ Geçerli tahmin yok\n")

# Sonuçları sakla
if results:
    results_df = pd.DataFrame(results)
    best_idx = results_df['RMSE'].idxmin()
    best_k = int(results_df.loc[best_idx, 'k'])
    
    print("\n" + "="*80)
    print("PARAMETRE OPTİMİZASYON SONUÇLARI (FULL TEST SET)")
    print("="*80)
    print(results_df.to_string(index=False))
    print(f"\n✓ En iyi k değeri: {best_k} (RMSE: {results_df.loc[best_idx, 'RMSE']:.4f})")
    print("="*80)
    
    # En iyi k değeriyle son modeli eğit
    print(f"\nFinal model k={best_k} ile eğitiliyor (tüm test seti için)...")
    knn_cf = ItemBasedKNN(n_neighbors=best_k, metric='cosine')
    knn_cf.fit(train_user_item)
    print("✓ Model eğitimi tamamlandı!")
else:
    print("Hiç sonuç alınamadı!")

KNN modeli eğitiliyor...
Test seti: 200042 örnek (tüm veri seti)

k = 3 ile eğitim başlıyor...
k = 3 tahminleri hesaplanıyor (200042 örnek)...
  50000 örnek işlendi... (49992 geçerli tahmin)
  100000 örnek işlendi... (99986 geçerli tahmin)
  150000 örnek işlendi... (149981 geçerli tahmin)
  200000 örnek işlendi... (199974 geçerli tahmin)

✓ k=3 Sonuçları:
   RMSE: 1.6636 | MAE: 1.2075 | R²: -1.2077
   Geçerli tahmin: 200016

k = 5 ile eğitim başlıyor...
k = 5 tahminleri hesaplanıyor (200042 örnek)...
  50000 örnek işlendi... (49992 geçerli tahmin)
  100000 örnek işlendi... (99986 geçerli tahmin)
  150000 örnek işlendi... (149981 geçerli tahmin)
  200000 örnek işlendi... (199974 geçerli tahmin)

✓ k=5 Sonuçları:
   RMSE: 1.4417 | MAE: 1.0295 | R²: -0.6580
   Geçerli tahmin: 200016

k = 7 ile eğitim başlıyor...
k = 7 tahminleri hesaplanıyor (200042 örnek)...
  50000 örnek işlendi... (49992 geçerli tahmin)
  100000 örnek işlendi... (99986 geçerli tahmin)
  150000 örnek işlendi... (149981 

In [7]:
# Test seti üzerinde tahminler yap (en iyi k ile) - HIZLANDIRILAN
print("\nTest seti üzerinde tahminler yapılıyor...")
print(f"İşleniyor: {len(test_sample)} örnek\n")

predictions = []
actuals = []
errors = []
processed = 0

for idx, row in test_sample.iterrows():
    user_id = row['UserID']
    movie_id = row['MovieID']
    actual_rating = row['Rating']
    
    # Sadece geçerli örnekleri işle
    if user_id in knn_cf.train_matrix.index and movie_id in knn_cf.train_matrix.columns:
        predicted_rating = knn_cf.predict(user_id, movie_id)
        predicted_rating = np.clip(predicted_rating, 1, 5)
        
        predictions.append(predicted_rating)
        actuals.append(actual_rating)
        errors.append(abs(predicted_rating - actual_rating))
        
        processed += 1
        
        # Her 1000 tahminde ilerlemesi göster
        if processed % 1000 == 0:
            print(f"  {processed} tahmin tamamlandı...")

predictions = np.array(predictions)
actuals = np.array(actuals)
errors = np.array(errors)

print(f"\n✓ Toplam tahmin sayısı: {len(predictions)}")
print(f"Ortalama tahmin: {predictions.mean():.3f}")
print(f"Ortalama actual: {actuals.mean():.3f}")


Test seti üzerinde tahminler yapılıyor...
İşleniyor: 200042 örnek

  1000 tahmin tamamlandı...
  2000 tahmin tamamlandı...
  3000 tahmin tamamlandı...
  4000 tahmin tamamlandı...
  5000 tahmin tamamlandı...
  6000 tahmin tamamlandı...
  7000 tahmin tamamlandı...
  8000 tahmin tamamlandı...
  9000 tahmin tamamlandı...
  10000 tahmin tamamlandı...
  11000 tahmin tamamlandı...
  12000 tahmin tamamlandı...
  13000 tahmin tamamlandı...
  14000 tahmin tamamlandı...
  15000 tahmin tamamlandı...
  16000 tahmin tamamlandı...
  17000 tahmin tamamlandı...
  18000 tahmin tamamlandı...
  19000 tahmin tamamlandı...
  20000 tahmin tamamlandı...
  21000 tahmin tamamlandı...
  22000 tahmin tamamlandı...
  23000 tahmin tamamlandı...
  24000 tahmin tamamlandı...
  25000 tahmin tamamlandı...
  26000 tahmin tamamlandı...
  27000 tahmin tamamlandı...
  28000 tahmin tamamlandı...
  29000 tahmin tamamlandı...
  30000 tahmin tamamlandı...
  31000 tahmin tamamlandı...
  32000 tahmin tamamlandı...
  33000 tahmi

In [8]:
# Performans metrikleri (en iyi model)
rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r_squared = 1 - (np.sum((actuals - predictions)**2) / np.sum((actuals - actuals.mean())**2))

# Coverage
coverage = (predictions != actuals.mean()).sum() / len(predictions) * 100

print("\n" + "="*60)
print("OPTİMİZE SONRASI PERFORMANS DEĞERLENDİRMESİ")
print("="*60)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE):      {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"R-squared (R²):                 {r_squared:.4f}")
print(f"Coverage:                       {coverage:.2f}%")
print("="*60)


OPTİMİZE SONRASI PERFORMANS DEĞERLENDİRMESİ
Root Mean Squared Error (RMSE): 1.0182
Mean Absolute Error (MAE):      0.7644
Mean Absolute Percentage Error (MAPE): 30.22%
R-squared (R²):                 0.1730
Coverage:                       100.00%


In [9]:
# Metrik karşılaştırması - Optimizasyon Öncesi vs Sonrası
print("\n" + "="*60)
print("OPTİMİZASYON ÖNCESİ vs SONRASI KARŞILAŞTIRMA")
print("="*60)

# İlk k=5 sonuçlarını (eski sonuçlar)
first_result = results_df[results_df['k'] == 5].iloc[0]
best_result = results_df.iloc[best_idx]

comparison_data = {
    'Metrik': ['RMSE', 'MAE', 'R²'],
    'Başlangıç (k=5)': [f"{first_result['RMSE']:.4f}", f"{first_result['MAE']:.4f}", f"{first_result['R2']:.4f}"],
    'Optimize (k={})'.format(int(best_k)): [f"{best_result['RMSE']:.4f}", f"{best_result['MAE']:.4f}", f"{best_result['R2']:.4f}"],
    'İyileşme': [
        f"{((first_result['RMSE'] - best_result['RMSE']) / first_result['RMSE'] * 100):.2f}%",
        f"{((first_result['MAE'] - best_result['MAE']) / first_result['MAE'] * 100):.2f}%",
        f"{((best_result['R2'] - first_result['R2']) / abs(first_result['R2']) * 100):.2f}%" if first_result['R2'] != 0 else "N/A"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print("="*60)


OPTİMİZASYON ÖNCESİ vs SONRASI KARŞILAŞTIRMA
Metrik Başlangıç (k=5) Optimize (k=30) İyileşme
  RMSE          1.4417          1.0182   29.37%
   MAE          1.0295          0.7644   25.75%
    R²         -0.6580          0.1730  126.29%


In [10]:
# DETAYLI HATA ANALİZİ (OPTİMİZE MODEL)
print("\nDETAYLI HATA ANALİZİ (OPTİMİZE MODEL)")
print("="*60)
print(f"Minimum MAE:                    {np.min(errors):.4f}")
print(f"Maximum MAE:                    {np.max(errors):.4f}")
print(f"Std Dev (Hata):                 {np.std(errors):.4f}")
print(f"Median MAE:                     {np.median(errors):.4f}")
print("\nTahmin dağılımı:")
print(f"  0.0-1.0 arası hata: {(errors < 1).sum() / len(errors) * 100:.2f}%")
print(f"  1.0-2.0 arası hata: {((errors >= 1) & (errors < 2)).sum() / len(errors) * 100:.2f}%")
print(f"  2.0+ arası hata:    {(errors >= 2).sum() / len(errors) * 100:.2f}%")


DETAYLI HATA ANALİZİ (OPTİMİZE MODEL)
Minimum MAE:                    0.0000
Maximum MAE:                    4.0000
Std Dev (Hata):                 0.6727
Median MAE:                     0.6003

Tahmin dağılımı:
  0.0-1.0 arası hata: 71.58%
  1.0-2.0 arası hata: 21.96%
  2.0+ arası hata:    6.46%


In [11]:
# RATING SINIFINA GÖRE PERFORMANS (OPTİMİZE MODEL)
print("\nRATING SINIFINA GÖRE PERFORMANS (OPTİMİZE MODEL)")
print("="*60)

test_results = pd.DataFrame({
    'Actual': actuals,
    'Predicted': predictions,
    'Error': errors
})

for rating in sorted(np.unique(actuals[actuals > 0])):
    mask = actuals == rating
    if mask.sum() > 0:
        rmse_by_rating = np.sqrt(mean_squared_error(actuals[mask], predictions[mask]))
        mae_by_rating = mean_absolute_error(actuals[mask], predictions[mask])
        count = mask.sum()
        print(f"Rating {rating}: RMSE={rmse_by_rating:.4f}, MAE={mae_by_rating:.4f}, Count={count}")


RATING SINIFINA GÖRE PERFORMANS (OPTİMİZE MODEL)
Rating 1: RMSE=1.9242, MAE=1.6669, Count=11394
Rating 2: RMSE=1.3976, MAE=1.2505, Count=21356
Rating 3: RMSE=0.8833, MAE=0.7339, Count=52111
Rating 4: RMSE=0.6842, MAE=0.4487, Count=69642
Rating 5: RMSE=1.0503, MAE=0.8283, Count=45513


In [14]:
# ÖZET RAPOR
print("\n" + "="*80)
print("ÖZETİ: OPTİMİZE SONRASI ITEM-BASED KNN COLLABORATİVE FİLTERİNG")
print("="*80)
print(f"Model Parametreleri:")
print(f"  - Algoritma: Item-Based KNN")
print(f"  - Komşu sayısı (k): {int(best_k)} (OPTİMİZE EDİLDİ)")
print(f"  - Mesafe metriği: Cosine")
print(f"\nVeri Seti Bilgisi:")
print(f"  - Eğitim örnekleri: {len(train_ratings)}")
print(f"  - Test örnekleri: {len(test_ratings)}")
print(f"  - Toplam kullanıcı: {ratings['UserID'].nunique()}")
print(f"  - Toplam film: {ratings['MovieID'].nunique()}")
print(f"\nOptimizasyon Bilgisi:")
print(f"  - Test edilen k değerleri: {list(results_df['k'].values)}")
print(f"  - En iyi k: {int(best_k)}")
print(f"\nPerformans Metrikleri (OPTİMİZE SONRASI):")
print(f"  - RMSE: {rmse:.4f}")
print(f"  - MAE: {mae:.4f}")
print(f"  - MAPE: {mape:.2f}%")
print(f"  - R²: {r_squared:.4f}")
print(f"  - Coverage: {coverage:.2f}%")
print("="*80)


ÖZETİ: OPTİMİZE SONRASI ITEM-BASED KNN COLLABORATİVE FİLTERİNG
Model Parametreleri:
  - Algoritma: Item-Based KNN
  - Komşu sayısı (k): 30 (OPTİMİZE EDİLDİ)
  - Mesafe metriği: Cosine

Veri Seti Bilgisi:
  - Eğitim örnekleri: 800167
  - Test örnekleri: 200042
  - Toplam kullanıcı: 6040
  - Toplam film: 3706

Optimizasyon Bilgisi:
  - Test edilen k değerleri: [3, 5, 7, 10, 15, 20, 30]
  - En iyi k: 30

Performans Metrikleri (OPTİMİZE SONRASI):
  - RMSE: 1.0182
  - MAE: 0.7644
  - MAPE: 30.22%
  - R²: 0.1730
  - Coverage: 100.00%
